In [426]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import itertools
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from scipy import stats
from scipy.optimize import curve_fit
import warnings
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
import seaborn.objects as so
warnings.filterwarnings("ignore", category=FutureWarning)

# DCA BACKTESTER

In [7]:
df = pd.read_csv('Binance_BTCUSDT_1min.csv')
df = df.iloc[:,:6]
df.columns=['timestamp','open', 'high', 'low', 'close', 'volume']
df.reset_index(drop=True, inplace=True)
df.timestamp = pd.to_datetime(df.timestamp)
df = df.set_index('timestamp')

In [8]:
def resample_df(df, freq):
    resampled_open = df.open.resample(freq).first()
    resampled_high = df.high.resample(freq).max()
    resampled_low = df.low.resample(freq).min()
    resampled_close = df.close.resample(freq).last()
    resampled_volume = df.volume.resample(freq).sum()
    new_df = pd.concat([resampled_open, resampled_high, resampled_low, resampled_close, resampled_volume], axis=1)
    new_df.dropna(inplace=True)
    return new_df

In [9]:
def calc_price(new_df):
    new_df['price'] = new_df.open.shift(-1)

In [10]:
def calc_rsi(new_df, rsi_length):
    new_df['rsi'] = ta.rsi(new_df.close, length = rsi_length)

In [11]:
def calc_buy_signal(new_df):
    new_df['buy_signal'] = np.where((new_df.rsi<30), True, False)

In [12]:
#prep_df:
new_df = resample_df(df, "1H")
calc_price(new_df)
calc_rsi(new_df, 9)
calc_buy_signal(new_df)

In [423]:
def backtest(df, freq, rsi_length, tp, base_order, price_deviation, so_amt, so_volume_mult, so_step_mult):

    new_df = resample_df(df, freq)
    calc_price(new_df)
    calc_rsi(new_df, rsi_length)
    calc_buy_signal(new_df)
    new_df.dropna(inplace=True)
    
    #Error handle for no buy signals
    if len(new_df[new_df.buy_signal > 0]) < 1:
        empty_result = pd.DataFrame({
            "entry_time": [0],
            "entry_price": [0],
            "tp_target": [0],
            "sl_target": [0],
            "exit_time": [0],
            "exit_price": [0],
            "pnl": [0],
            "equity": [0],
            "pnl_perc": [0]
        })
        amount = 0
        winrate = 0
        pnl = 0
        equity_float = 0
        #return amount, winrate, pnl_perc, equity_float
        return empty_result
    
    #Initialise Varibles
    in_position = False
    trades = []
    buys = []
    current_trade = {}
    
  

    '''#bot presets
    base_order = 100
    price_deviation = 0.02
    so_amt = 4
    so_volume_mult = 2
    so_step_mult = 2'''

    
    initial_equity = 1000
    equity = initial_equity

    quote_pnl_value = 0
    base_pnl_value = 0

    filled_stages = set()
    order_prices = []
            
    for i in range(len(new_df)-1):
    #Check exit conditions
        if in_position:
            if any(new_df.iloc[i].low < order_price for order_price in order_prices):
                if (len(filled_stages) != len(order_prices)):
                    for j, (order_price, order_volume) in enumerate(zip(order_prices, order_volumes), 1):
                        if j not in filled_stages:
                            if new_df.iloc[i].low< order_price:
                                current_trade["base_amt"] = current_trade["base_amt"] + order_volume / order_price
                                current_trade["quote_size"] = current_trade["quote_size"] + order_volume
                                current_trade["stage"] = j
                                filled_stages.add(j)

            elif new_df.iloc[i].high > current_trade["tp_price"]:
                current_trade["exit_price"] = current_trade["tp_price"]
                base_pnl = current_trade['base_amt'] - (current_trade['quote_size']/current_trade['exit_price'])
                quote_pnl = current_trade['quote_size']*(tp-1)
                pnl_perc = tp - 1
                base_pnl_value += base_pnl
                quote_pnl_value += quote_pnl
                #equity = equity + (equity * pnl_perc)
                #equity_str = "{:.2f}".format(equity)
                trades.append({
                    "entry_time":current_trade["entry_time"],
                    "entry_price":current_trade["entry_price"],
                    "tp_target":current_trade["tp_price"],
                    "exit_time":new_df.iloc[i].name,
                    "exit_price":current_trade["exit_price"], #changed this line
                    "base_amt":current_trade["base_amt"],
                    "quote_size":current_trade["quote_size"],
                    "stage":current_trade["stage"],
                    "pnl_perc": pnl_perc,
                    
                    #"equity": equity,
                    "base_profit":base_pnl,
                    "quote_profit":quote_pnl,
                    "cum_base_pnl":base_pnl_value,
                    "cum_quote_pnl":quote_pnl_value,
                })
                current_trade = {}
                filled_stages.clear()  # Clear the set to make it empty for the next iteration
                order_prices = []
                in_position = False

        #Check entry conditions
        if not in_position:
            if new_df.iloc[i].buy_signal == True:
                current_trade["entry_time"] = new_df.iloc[i+1].name
                current_trade["entry_price"] = new_df.iloc[i].price
                current_trade["tp_price"] = new_df.iloc[i].price*tp
                current_trade["base_amt"] = base_order/new_df.iloc[i].price
                current_trade["quote_size"] = base_order
                current_trade["avg_price"] = current_trade["quote_size"]/current_trade["base_amt"]
                current_trade["stage"] = 0
                current_trade["base_profit"] = 0
                current_trade["quote_profit"] = 0
                in_position = True
                order_prices = [current_trade['entry_price'] * (1 - price_deviation * (so_step_mult ** (i - 1))) for i in range(1, so_amt + 1)]
                for order_price in order_prices:
                    current_trade[f'order_{order_prices.index(order_price)}'] = order_price
                order_volumes = [base_order * (so_volume_mult ** (i-1)) for i in range(1, so_amt +1)]
                #print(f'took trade at {current_trade["entry_price"]}')
                
                
    data = pd.DataFrame(trades)
    amount = len(data)
    #winrate = round(len(data.loc[data.pnl.values>0])/len(data)*100,2)
    #cum_pnl_perc = round(sum(pd.Series(data.pnl_perc))*100,2)
    #equity_value = round(data.equity[len(data) - 1], 2)
    #pnl = round(float(data.equity[len(data) - 1]) - initial_equity,2)
    length = data.exit_time[len(data)-1] - data.entry_time[0]
    #max_drawdown = 100 - data.equity.min()/initial_equity*100
    #equity_high = round(data.equity.max(),2)
    #equity_low = round(data.equity.min(),2)
    
    """print(f"Winrate: {winrate}%")
    print(f"Amount of trades: {amount}")
    print(f"Culmulative pnl: {cum_pnl_perc}%")
    print("")
    print(f"Max drawdown: {max_drawdown} %")
    print(f"Equity Max: {equity_high} USD")
    print(f"Equity Min: {equity_low} USD")
    print(f"Final equity value: {equity_value} USD")
    print("")
    print(f"Summary: ${pnl} profit made from ${initial_equity} initial in about {length} hours & minutes")"""
    #return winrate, amount, cum_pnl_perc, max_drawdown, equity_value
    return data

In [424]:
#def backtest(df, freq, rsi_length, tp, base_order, price_deviation, so_amt, so_volume_mult, so_step_mult):

In [425]:
backtest(df, "30T", 14, 1.02, 100, 0.04, 5, 1.5, 3)

,entry_time,entry_price,tp_target,exit_time,exit_price,base_amt,quote_size,stage,pnl_perc,base_profit,quote_profit,cum_base_pnl,cum_quote_pnl
0,2017-08-18 19:00:00,4086.28,4168.0056,2017-08-19 00:30:00,4168.0056,0.024472,100.0,0,0.02,0.000480,2.0,0.000480,2.0
1,2017-08-21 06:30:00,3991.05,4070.8710,2017-08-21 09:00:00,4070.8710,0.025056,100.0,0,0.02,0.000491,2.0,0.000971,4.0
2,2017-08-22 02:00:00,3700.00,3774.0000,2017-08-22 02:00:00,3774.0000,0.027027,100.0,0,0.02,0.000530,2.0,0.001501,6.0
3,2017-08-22 05:00:00,3500.80,3570.8160,2017-08-22 05:00:00,3570.8160,0.028565,100.0,0,0.02,0.000560,2.0,0.002061,8.0
4,2017-08-28 04:30:00,4160.60,4243.8120,2017-08-28 05:00:00,4243.8120,0.024035,100.0,0,0.02,0.000471,2.0,0.002532,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,2021-10-31 10:00:00,60666.39,61879.7178,2021-11-01 07:00:00,61879.7178,0.001648,100.0,0,0.02,0.000032,2.0,0.089373,348.0
119,2021-11-03 14:00:00,61821.07,63057.4914,2021-11-03 18:30:00,63057.4914,0.001618,100.0,0,0.02,0.000032,2.0,0.089405,350.0
120,2021-11-10 21:30:00,64860.03,66157.2306,2021-11-15 00:00:00,66157.2306,0.001542,100.0,0,0.02,0.000030,2.0,0.089435,352.0
121,2021-11-15 16:00:00,64288.16,65573.9232,2024-03-04 13:30:00,65573.9232,0.011296,575.0,3,0.02,0.002527,11.5,0.091962,363.5
